In [50]:
from openai import OpenAI
import os

In [51]:
client = OpenAI(api_key="")

In [52]:
uploaded_file = client.files.create(
    file=open("Welcome to our extensive MLOps Bootcamp.pdf", 'rb'),
    purpose='assistants'
)

In [53]:
uploaded_file

FileObject(id='file-8bh4216zywlokjky1vmeVhWM', bytes=350109, created_at=1720025888, filename='Welcome to our extensive MLOps Bootcamp.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [54]:
# Step 1: Create an Assistant
assistant = client.beta.assistants.create(
  name="Course Browser",
  instructions="You are an assistant to provide details about the course.",
  model="gpt-3.5-turbo",
  tools=[{"type": "retrieval"}],
    file_ids=[uploaded_file.id]
)

In [55]:
assistant

Assistant(id='asst_P4IkyVuhIGNiix2Spx9kX1ds', created_at=1720025889, description=None, file_ids=['file-8bh4216zywlokjky1vmeVhWM'], instructions='You are an assistant to provide details about the course.', metadata={}, model='gpt-3.5-turbo', name='Course Browser', object='assistant', tools=[ToolRetrieval(type='retrieval')], top_p=1.0, temperature=1.0, response_format='auto')

In [56]:
thread = client.beta.threads.create()
thread

Thread(id='thread_47btdYAclnDo0qc2VQ8O9b3h', created_at=1720025891, metadata={}, object='thread')

In [57]:
message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role='user',
            content="What is the content of 'Packaging the ML models'."
)

In [58]:
run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
            instructions="Help the user on course query."
            )
run    

Run(id='run_C880pR7NYVRGy063KzsIPSEm', assistant_id='asst_P4IkyVuhIGNiix2Spx9kX1ds', cancelled_at=None, completed_at=None, created_at=1720025892, expires_at=1720026492, failed_at=None, file_ids=['file-8bh4216zywlokjky1vmeVhWM'], instructions='Help the user on course query.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_47btdYAclnDo0qc2VQ8O9b3h', tools=[ToolAssistantToolsRetrieval(type='retrieval')], usage=None, temperature=1.0, top_p=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto', parallel_tool_calls=True)

In [59]:
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == 'completed':
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            latest_message = messages.data[0]
            text = latest_message.content[0].text.value
            print(text)
            break

The section on "Packaging the ML Models" covers the following points:
- Delving into the art of packaging Machine Learning models.
- Exploring different packaging techniques and their implications.
- Ensuring that ML models are easily deployable and reproducible【5†source】.
